# Hackster - AMD AI project

This project utilises Microsoft Phi-3 model to manage robotics navigation.

In [ ]:
# Importing required libraries
import re
import onnxruntime_genai as og
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

In [ ]:
# Loading Phi-3 ONNX model
model = og.Model("<PATH_TO_ONNX_MODEL>")

tokenizer = og.Tokenizer(model)
tokenizer_stream = tokenizer.create_stream()

search_options = {"max_length": 3072,"temperature": 0}
params = og.GeneratorParams(model)
params.set_search_options(**search_options)

In [ ]:
# Helper function for System 2 logic with Phi-3 model
def system2_logic(distance_left, distance_right):
    prompt = f"<|system|><YOUR_SPECIFIC_PROMPT><|end|><|assistant|>"
    input_tokens = tokenizer.encode(prompt)
    params.input_ids = input_tokens
    response = ""

    generator = og.Generator(model, params)

    while not generator.is_done():
        generator.compute_logits()
        generator.generate_next_token()

        new_token = generator.get_next_tokens()[0]
        response += tokenizer_stream.decode(new_token)

    del generator

    return response

In [ ]:
# Helper function to format the response from Phi-3 model
def extract_first_list(data):
    # Join the list into a single string
    data_str = ''.join(data)
    
    # Use regex to find the first list pattern
    match = re.search(r'\[\s*-?\d+,\s*-?\d+\]', data_str)
    
    if match:
        # Extract the matched string
        list_str = match.group(0)
        
        # Convert the string to a list of integers
        list_values = eval(list_str)
        
        return list_values
    else:
        return None

In [ ]:
# Initiating CoppeliaSim remote API client and robot's sensors
print("Starting CoppeliaSim simulation...")
client = RemoteAPIClient()
sim = client.require("sim")

leftMotorHandle = sim.getObject("/PioneerP3DX/leftMotor")
rightMotorHandle = sim.getObject("/PioneerP3DX/rightMotor")
proximitySensorHandle = sim.getObject("/PioneerP3DX/Proximity_sensor_Centre")
proximitySensorHandleLeft = sim.getObject("/PioneerP3DX/Proximity_sensor_Left")
proximitySensorHandleRight = sim.getObject("/PioneerP3DX/Proximity_sensor_Right")

sim.setJointTargetVelocity(leftMotorHandle, 0)
sim.setJointTargetVelocity(rightMotorHandle, 0)
sim.startSimulation()

In [ ]:
# Managing robotic system
startTime = sim.getSimulationTime()
while sim.getSimulationTime() - startTime < 600:
    # Check for obstacles    
    response, distance, _, _, _ = sim.handleProximitySensor(proximitySensorHandle)

    if response == 1:
        print(f"Obstacle detected at a distance of {distance:.2f} meters!")
        sim.setJointTargetVelocity(leftMotorHandle, 0)
        sim.setJointTargetVelocity(rightMotorHandle, 0)
           
        # System 2 logic
        _, distance_left, _, _, _ = sim.handleProximitySensor(proximitySensorHandleLeft)
        _, distance_right, _, _, _ = sim.handleProximitySensor(proximitySensorHandleRight)
        phi3_recommendations = system2_logic(distance_left, distance_right)
        phi3_recommendations = extract_first_list(phi3_recommendations)
        print("Phi-3 recommendations:", phi3_recommendations)
        try:
            left_wheel_speed = list(phi3_recommendations)[0]
            right_wheel_speed = list(phi3_recommendations)[1]
        except:
            left_wheel_speed = 0
            right_wheel_speed = 0
        print(f"Submitted instruction: Left wheel speed: {left_wheel_speed}, Right wheel speed: {right_wheel_speed}")
        sim.setJointTargetVelocity(leftMotorHandle, left_wheel_speed)
        sim.setJointTargetVelocity(rightMotorHandle, right_wheel_speed)

    else:
        # System 1 logic
        sim.setJointTargetVelocity(leftMotorHandle, 1)
        sim.setJointTargetVelocity(rightMotorHandle, 1)
    
    sim.resetProximitySensor(proximitySensorHandle)
    sim.resetProximitySensor(proximitySensorHandleLeft)
    sim.resetProximitySensor(proximitySensorHandleRight)

# Stopping the simulation
sim.stopSimulation()
print('Program ended')